<a href="https://colab.research.google.com/github/nazanaza2970/BENTO-PACKAGING-ACTIVITY-RECOGNITION-CHALLENGE/blob/naza/Bento_Quaternion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from tqdm.notebook import trange, tqdm
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score

# Take 2

In [3]:
def get_quaternion(lst1,lst2,matchlist=None):
    if not matchlist:
        matchlist=range(len(lst1))
    M=np.matrix([[0,0,0],[0,0,0],[0,0,0]])

    for i,coord1 in enumerate(lst1):
        x=np.matrix(np.outer(coord1,lst2[matchlist[i]]))
        M=M+x

    N11=float(M[0][:,0]+M[1][:,1]+M[2][:,2])
    N22=float(M[0][:,0]-M[1][:,1]-M[2][:,2])
    N33=float(-M[0][:,0]+M[1][:,1]-M[2][:,2])
    N44=float(-M[0][:,0]-M[1][:,1]+M[2][:,2])
    N12=float(M[1][:,2]-M[2][:,1])
    N13=float(M[2][:,0]-M[0][:,2])
    N14=float(M[0][:,1]-M[1][:,0])
    N21=float(N12)
    N23=float(M[0][:,1]+M[1][:,0])
    N24=float(M[2][:,0]+M[0][:,2])
    N31=float(N13)
    N32=float(N23)
    N34=float(M[1][:,2]+M[2][:,1])
    N41=float(N14)
    N42=float(N24)
    N43=float(N34)

    N=np.matrix([[N11,N12,N13,N14],\
              [N21,N22,N23,N24],\
              [N31,N32,N33,N34],\
              [N41,N42,N43,N44]])


    values,vectors=np.linalg.eig(N)
    w=list(values)
    mw=max(w)
    quat= vectors[:,w.index(mw)]
    quat=np.array(quat).reshape(-1,).tolist()
    return quat

In [9]:
get_quaternion([1,2], [2,4,6])

[0.0, 0.5773502691896258, 0.5773502691896257, 0.5773502691896258]

In [10]:
import time
t1 = time.time()
for _ in range(10000):
    get_quaternion([1,2], [2,4,6])
print(time.time() - t1)

5.356082916259766


# Take 1

In [6]:
data = pd.read_pickle("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/windowed_500_xz",compression = "xz")

In [16]:
vectors = []
vectors.append(list(data["X4"]-data["X7"]))
vectors.append(list(data["Y4"]-data["Y7"]))
vectors.append(list(data["Z4"]-data["Z7"]))

vectors.append(list(data["X13"]-data["X9"]))
vectors.append(list(data["Y13"]-data["Y9"]))
vectors.append(list(data["Z13"]-data["Z9"]))

vectors.append(list(data["X13"]-data["X6"]))
vectors.append(list(data["Y13"]-data["Y6"]))
vectors.append(list(data["Z13"]-data["Z6"]))

In [17]:
vector = np.transpose(np.array(vectors))

In [9]:
vector[:,0:3].shape

(1786500, 3)

In [10]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

In [12]:
def diffVectors2(v1, v2):
    """ Get rotation Quaternion between 2 vectors """
    v1 = normalize(v1)
    v2 = normalize(v2)
    v = v1+v2
    v = normalize(v)
    angle = np.dot(v1,v2)
    axis = np.cross(v1,v2)
    # print(axis)
    # print(angle)
    return angle, axis[0],axis[1],axis[2] 

In [ ]:
quaternions = []
for i in tqdm(range(0,9,3)):
    a = vector[:,i:i+3]
    quats = []
    for i in tqdm(range(0,len(a)-1)):
        quats.append(np.array(diffVectors2(a[i],a[i+1])))
    quaternions.append(quats)

In [ ]:
pd.DataFrame(quaternions).to_pickle("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/quats2_xz.pkl",compression = "xz")